Важное:
1. Один скрипт должен считать входной корпус из файла (ов), обработать его и в каком-то формате записать «модель» в другой файл
2. Другой скрипт должен принимать на вход путь до этой модели и опционально префикс генерируемого текста и генерировать его правдоподобное продолжение.
3. Основной код должен быть разбит на две части: обучение и генерация.
4. Удобно создать для обучения и генерации отдельные файлы и реализовать консольный интерфейс к ним через argparse

Дополнительное:
1. Считать входные данные из файлов.
2. Очистить тексты: выкидывать неалфавитные символы, приводить к lowercase.
3. Разбить тексты на слова (в ML это называется токенизацией).
4. Сохранить модель в каком-нибудь формате, который позволяет восстановить слова и частоты биграмм.
5. Загрузить модель.
6. Инициализировать её каким-нибудь сидом.
7. Сгенерировать последовательность нужной длины.
8. Следуйте принципам ООП. Оберните модель в класс, у которого будет методы fit и generate.
9. Для сохранения модели удобно использовать pickle или dill.


In [401]:
import math 
import numpy as np

TrainLines = []
Words = []
AssessmentTest = []
SentLength = []
ConfusionCoef = []

Dictionary = {'<s>': 0, '</s>': 0, '<NN>': 1}
BiDictionary = {'<NN> <NN>':0}
BiProbs = {}

for line in open('/content/text.txt'):
    if line.strip():
        TrainLines.append(line.strip())
TrainLines = [line.strip() for line in open('/content/test.txt')]

for line in TrainLines:
    Dictionary['<s>'] += 1
    Words.append('<s>') 
    Dictionary['</s>'] += 1

    for word in line.split():       
        Words.append(word)
        
        if word not in Dictionary: Dictionary[word] = 1
        else: Dictionary[word] += 1
    Words.append('</s>')
  
Vocabulary = set(Dictionary)
listVocab = list(Vocabulary)

CountTokens = sum(Dictionary.values())
print('------------------------------------')
print('Words for train', len(Words))
print('Vocabulary size:', len(Vocabulary))
print('Vocabulary examples:', listVocab[0:2])
print('------------------------------------')

for Word1, Word2 in zip(Words, Words[1:]): 
    OneBi = Word1 + ' ' + Word2
    if OneBi not in BiDictionary: BiDictionary[OneBi] = 1
    else: BiDictionary[OneBi] += 1

for Key in BiDictionary: BiDictionary[Key] += 1
print('Size BiDictionary:', len(BiDictionary))

for Key in BiDictionary:
    Word1, Word2 = Key.split()
    probability = BiDictionary[Key] / (Dictionary[Word1] + len(Vocabulary))
    BiProbs[Key] = probability
        
print('Size BiProbs:', len(BiProbs))
vocabularySize = len(Dictionary)

print('------------------------------------')
for line in TrainLines:
    print('Line',TrainLines.index(line) + 1,': ', line, '\n')
    words = line.split()
    words.insert(0, '<s>')
    words.append('</s>')
    
    SentLength.append(len(words))
    
    BiTest = []
    for Word1, Word2 in zip(words,words[1:]): 
        OneBi = Word1 + ' ' + Word2
        BiTest.append(OneBi)
    print('Bigrams:\n', BiTest, '\n')
    
    Product = 1
    
    for i in range(len(BiTest)):
        firstWordProbability = 1
        CurrentBi = BiTest[i]
                  
        if CurrentBi not in BiProbs: CurrentBi = '<NN> <NN>'
        Product = Product * BiProbs[CurrentBi] 
        
    print('Product:', Product, '\n')
    AssessmentTest.append(Product)

print('------------------------------------')
print('Probabilties for test:', AssessmentTest)
print('Sentence lengths:', SentLength)

normalizedBySentenceLength = []

for i in range(len(AssessmentTest)):
    normalizedProb1 = math.exp(math.log(AssessmentTest[i]) / SentLength[i])
    normalizedBySentenceLength.append(normalizedProb1)
    
#print('P normalized by sentence length:', normalizedBySentenceLength)

for i in range(len(AssessmentTest)):
    perplexity = (1 / AssessmentTest[i]) ** (1/SentLength[i])
    ConfusionCoef.append(perplexity)
  
print('------------------------------------')
print('Confusion Coefficient:', ConfusionCoef)
OUTPUT = open('output.txt','w') 

for i in range(len(AssessmentTest)): OUTPUT.write(str(AssessmentTest[i]) + ',' + str(normalizedBySentenceLength[i]) + ',' + str(ConfusionCoef[i]) + '\n')
OUTPUT.close() 

YourSentence = '<s>'
CurrentWord = '<s>'
while CurrentWord != '</s>' and CurrentWord != '.' and CurrentWord != '?': 
    PossibleNextWord = []
    for Key in BiDictionary:
        Word1, Word2 = Key.split()
        if Word1 == CurrentWord:
            PossibleNextWord.append(Word2)

    OneArray = np.array(PossibleNextWord) 
    NextWord = np.random.choice(a = OneArray) 
    YourSentence = YourSentence + ' ' + NextWord
    CurrentWord = NextWord

------------------------------------
Words for train 12
Vocabulary size: 7
Vocabulary examples: ['<s>', 'должен']
------------------------------------
Size BiDictionary: 10
Size BiProbs: 10
------------------------------------
Line 1 :  Здесь 

Bigrams:
 ['<s> Здесь', 'Здесь </s>'] 

Product: 0.045454545454545456 

Line 2 :  должен 

Bigrams:
 ['<s> должен', 'должен </s>'] 

Product: 0.045454545454545456 

Line 3 :  быть 

Bigrams:
 ['<s> быть', 'быть </s>'] 

Product: 0.045454545454545456 

Line 4 :  текст 

Bigrams:
 ['<s> текст', 'текст </s>'] 

Product: 0.045454545454545456 

------------------------------------
Probabilties for test: [0.045454545454545456, 0.045454545454545456, 0.045454545454545456, 0.045454545454545456]
Sentence lengths: [3, 3, 3, 3]
------------------------------------
Confusion Coefficient: [2.802039330655387, 2.802039330655387, 2.802039330655387, 2.802039330655387]
<s> быть </s>


In [402]:
print(YourSentence)

<s> быть </s>
